<a href="https://colab.research.google.com/github/Tinynja/Sarsa-phi-EB/blob/main/ALE_Framework_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
try:
    # Copy GitHub workspace into Colab
    import google.colab
    !git clone https://github.com/Tinynja/Sarsa-phi-EB
    !mv Sarsa-phi-EB/* .
except:
    print('Skipping GitHub cloning since not running in Colab.')

Skipping GitHub cloning since not running in Colab.


In [4]:
# Built-in libraries

# Pipy libraries


In [3]:
# Import all supported ROMs into ALE
!ale-import-roms ROMS\

[SUPPORTED]           tic_tac_toe_3d ROMS\3-D Tic-Tac-Toe (1980) (Atari, Carol Shaw - Sears) (CX2618 - 49-75123) ~.bin
[SUPPORTED]                adventure ROMS\Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin
[SUPPORTED]                 air_raid ROMS\Air Raid (Men-A-Vision) (PAL) ~.bin
[SUPPORTED]                    alien ROMS\Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin
[SUPPORTED]                   amidar ROMS\Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin
[SUPPORTED]                  assault ROMS\Assault (AKA Sky Alien) (1983) (Bomb - Onbase) (CA281).bin
[SUPPORTED]                  asterix ROMS\Asterix (AKA Taz) (07-27-1983) (Atari, Jerome Domurat, Steve Woita) (CX2696) (Prototype).bin
[SUPPORTED]                  asterix ROMS\Asterix (AKA Taz) (1984) (Atari, Jerome Domurat, Steve Woita) (CX2696).bin
[SUPPORTED]                asteroids ROMS\Asteroids (1981) (Atari, Brad Stewart - Sears) (CX2649 - 49-751

[NOT SUPPORTED]                      ROMS\Hot Wave (AKA Ram It) (Double-Game Package) (1983) (Otto Versand) (746422) (PAL).bin
[NOT SUPPORTED]                      ROMS\Oscar's Trash Race (Kid's Controller) (03-30-1983) (Atari - CCW, Christopher H. Omarzu, Preston Stuart, Bruce Williams) (CX26101) (Prototype).bin
[NOT SUPPORTED]                      ROMS\Bowling (208 in 1) (Unknown) (PAL).bin
[NOT SUPPORTED]                      ROMS\Dukes of Hazzard (1983) (Atari, Mark R. Hahn) (CX2678) ~.bin
[NOT SUPPORTED]                      ROMS\Confrontation (1983) (Answer Software Corporation - TY Associates, Kim Ellis) (ASC2001) (Prototype) ~.bin
[NOT SUPPORTED]                      ROMS\Mr. Do!'s Castle (1984) (Parker Brothers) (PB5820) (Prototype).bin
[NOT SUPPORTED]                      ROMS\RealSports Baseball (1982) (Atari, Eric Manghise, Mimi Nyden, Joseph Tung) (CX2640) (Prototype).bin
[NOT SUPPORTED]                      ROMS\Fussball (AKA International Soccer) (Videospielkassette - Ar

[NOT SUPPORTED]                      ROMS\Bob Is Going Home (AKA Bobby Is Going Home) (JVP).bin
[NOT SUPPORTED]                      ROMS\Power Play Arcade Video Game Album V - Mogul Maniac, Surf's Up, Off Your Rocker, S.A.C. Alert (1984) (Amiga) (1130) (Prototype).bin
[NOT SUPPORTED]                      ROMS\Halloween (1983) (Wizard Video Games - MicroGraphic Image, Robert Barber, Tim Martin) (007) (Prototype) [a].bin
[NOT SUPPORTED]                         ROMS\Astrowar (Unknown).bin
[NOT SUPPORTED]                           ROMS\Frisco (Unknown).bin
[NOT SUPPORTED]                      ROMS\Deadly Duck (1982) (20th Century Fox Video Games - Sirius Software, Ed Hodapp) (11004) ~.bin
[NOT SUPPORTED]                      ROMS\Kangaroo (01-19-1983) (Atari - GCC, Kevin Osborn) (CX2689) (Prototype).bin
[NOT SUPPORTED]                      ROMS\Joust (08-09-1983) (Atari - GCC, Mike Feinstein, Kevin Osborn) (CX2691) (Prototype).bin
[NOT SUPPORTED]                      ROMS\Jaw Breaker (198

[NOT SUPPORTED]                      ROMS\Big Bird's Egg Catch (Grover's Egg Catch) (Kid's Controller) (12-08-1982) (Atari - CCW, Christopher H. Omarzu) (CX26104) (Prototype).bin
[NOT SUPPORTED]                      ROMS\International Soccer (1982) (M Network - APh Technological Consulting, Kevin Miller) (MT5687) ~.bin
[NOT SUPPORTED]                      ROMS\Pumuckl I (AKA Panda Chase) (1983) (ITT Family Games) (PAL).bin
[NOT SUPPORTED]                      ROMS\Survival Island (Jungle Raid) (1983) (Starpath Corporation, Steve Mundry, Scott Nelson) (12) (AR-4401) ~.bin
[NOT SUPPORTED]                      ROMS\Demon Attack (Canal 3 - Intellivision) (C 3016).bin
[NOT SUPPORTED]                      ROMS\Astrowar (Astrobattle) (Dimax - Sinmax) (SM8002) (PAL) ~.bin
[NOT SUPPORTED]                      ROMS\Dumbo's Flying Circus (Dumbo Flies Home) (1983) (Atari, Jerome Domurat, Peter C. Niday) (CX26115) (Prototype) (PAL).bin
[NOT SUPPORTED]                      ROMS\Kick-Man (01-08-82) (

In [7]:
import os
os.getcwd()

'D:\\Dev\\Sarsa-phi-EB'